# Importando bibliotecas

In [1]:
import logging
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import pandas as pd
from stable_baselines3 import DQN
from stable_baselines3.dqn.policies import MlpPolicy
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import time
from datetime import datetime
from torch import nn  # Import the neural network module from PyTorch
import gym
from gym import spaces
from sklearn.model_selection import train_test_split
from stable_baselines3 import DQN
from stable_baselines3.dqn.policies import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from sklearn.utils import resample
import sys
import os
sys.path.append(os.path.join('..'))
from classes._exploration import exploration
from classes._Env3WGym import Env3WGym
from classes._LoadInstances import LoadInstances
from classes._Agent import Agent



In [2]:
def validation_dqn(accuracy, dataset_validation_scaled, dqn_model, event_name):
       
    if accuracy > 0.8:
       
        logging.info(f'Iniciando a separação dos grupos de dados para validação individual')
        # Obtendo os índices que ordenariam a primeira coluna
        sort_indices = np.argsort(dataset_validation_scaled[:, 0])

        # Usando esses índices para reordenar todo o array
        dataset_validation_sorted = dataset_validation_scaled[sort_indices]
        
        # Inicializando a lista para armazenar os sub-datasets
        datasets = []
        current_dataset = []

        # Inicializando previous_datetime como None para a primeira comparação
        previous_datetime = None

        for row in dataset_validation_sorted:
            current_datetime = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S')
            
            # Verifica se é a primeira iteração ou se a diferença é maior que 1 hora
            if previous_datetime is None or (current_datetime - previous_datetime).total_seconds() / 3600 > 1:
                # Se não for a primeira iteração e a condição for verdadeira, inicia um novo dataset
                if current_dataset:
                    datasets.append(np.array(current_dataset))
                    current_dataset = []
            
            # Adiciona o registro atual ao dataset corrente
            current_dataset.append(row)
            previous_datetime = current_datetime

        # Não esqueça de adicionar o último dataset após terminar o loop
        if current_dataset:
            datasets.append(np.array(current_dataset))        

        logging.info(f'Fim da separação dos grupos de dados para validação com {len(datasets)} grupos de instâncias')
        
        count = -1
        acc_total = []
        array_prec_total = []
        for dataset_test in datasets:
            acc = 0
            count += 1
            logging.info(f'Iniciando predição da {count}ª instância de validação usando DQN')
            array_action_pred = []
            for i in range(0, len(dataset_test)):
                obs = dataset_test[i, 1:-1].astype(np.float32)
                action, _states = dqn_model.predict(obs, deterministic=True)  
                array_action_pred.append(action)

                true_action = dataset_test[i, -1]
                if true_action == 0:
                    acc +=  1 if action == 0 else 0
                elif true_action in range(1, 10):
                    acc +=  1 if action == 1 else 0
                elif true_action in range(101, 110):  # Corrigido para refletir o intervalo correto
                    acc +=  1 if action == 1 else 0  
                
                    
            acc_total.append(acc)
            array_prec_total.append(len(array_action_pred))        
            final_acc = int(acc)/len(array_action_pred) * 100
            
            expanded_array = np.column_stack((dataset_test, array_action_pred))
              
        
            
            df = pd.DataFrame(expanded_array, columns = ['timestamp', 'P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'class', 'action'])
            df.set_index('timestamp', inplace=True)
            df[['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP']] = df[['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP']].astype('float32')
            df['class'] = df['class'].astype(float).astype('int16')
            df['action'] = df['action'].astype(float).astype('int16')

            # Faça um filtro no dataframe df para exibir apenas os registros cujo o valor da coluna class é igual a 0

            numerator_normal = len(df[(df['class'] == 0) & (df['action'] == 0)])
            denominator_normal = len(df.loc[df['class'] == 0])
            acc_normal = numerator_normal / denominator_normal if denominator_normal > 0 else 0

            numerator_falha = len(df[(df['class'] != 0) & (df['action'] == 1)])
            denominator_falha = len(df.loc[df['class'] != 0])
            acc_falha = numerator_falha / denominator_falha if denominator_falha > 0 else 0

            logging.info(f'Acurácia da {count}ª instância: {final_acc:.3f}%')
            print(f'Acurácia da {count}ª instância: {final_acc:.3f}%')
            logging.info(f'Acurácia de Não-Falha na {count}ª instância: {acc_normal * 100:.3f}%')
            logging.info(f'Acurácia das Falha na {count}ª instância: {acc_falha * 100:.3f}%')
            #logging.info(f'Acurácia de Detecção de Falhas na {count}ª instância : {acc_ref * 100:.3f}%')
            logging.info(f'Fim predição da instância de teste DQN')  

            additional_labels = [
                f'Acurácia (Teste): {accuracy * 100:.1f}%', 
                f'Acurácia (Validação): {final_acc:.1f}%',  
                f'Acurácia de Não-Falha: {acc_normal * 100:.1f}%',  
                f'Acurácia de Falha: {acc_falha * 100:.1f}%' 
            ]

            explora = exploration(df)
            explora.plot_sensor(sensor_columns = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP'], _title = f'[{count}] - {event_name} - DQN', additional_labels =  additional_labels)
        
        logging.info(f'Acurácia: {sum(acc_total)/sum(array_prec_total) * 100:.3f}% no conjunto de dados de validação usando DQN')
        print(f'Acurácia: {sum(acc_total)/sum(array_prec_total) * 100:.3f}% no conjunto de dados de validação usando DQN')

    else:
        logging.info(f'Acurácia insuficiente para validação individual')
        print(f'Acurácia insuficiente para validação individual')

In [3]:
if __name__ == '__main__':

    events_names = {
        0: 'Normal',
        # 1: 'Abrupt Increase of BSW',
        # 2: 'Spurious Closure of DHSV',
        # 3: 'Severe Slugging',
        4: 'Flow Instability',
        # 5: 'Rapid Productivity Loss',
        # 6: 'Quick Restriction in PCK',
        # 7: 'Scaling in PCK',
        # 8: 'Hydrate in Production Line'
    }

    event_name = [value for key, value in events_names.items() if key != 0][0]
    
    
    directory = f'..\\..\\logs\\{event_name}'
    path_dataset = '..\\..\\..\\dataset'   
    path_model = f'..\\models\\{event_name}'  

    if not os.path.exists(directory):
        os.makedirs(directory)
        

    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_filename = f'{directory}\\{current_time}_{event_name}-log.txt'
    # Configuração do Logging
    logging.basicConfig(filename=log_filename, filemode='w', level=logging.INFO, format='[%(levelname)s]\t%(asctime)s - %(message)s', datefmt='%d/%m/%Y %I:%M:%S %p', force=True, encoding='utf-8')

    instances = LoadInstances(path_dataset)
    
    logging.info(f'Iniciando carregamento do dataset')
    dataset = instances.load_instance_with_numpy(events_names)    
    logging.info(f'Fim carregamento do dataset')
    
    logging.info(f'Iniciando divisão do dataset em treino e teste')
        
    # Definindo a porcentagem para divisão entre treino e teste
    train_percentage = 0.8  # 80% para treino

    dataset_train_scaled, dataset_test_scaled, dataset_validation_scaled = instances.data_preparation(dataset, train_percentage)

    '''logging.info(f'Iniciando treinamento do algoritmo DQN')    
    start_time = time.time()
    agente = Agent(path_model)
    agente.env3W_dqn(dataset_train_scaled, n_envs = 5)  
    print(f"Tempo de Treinamento DQN: {round(time.time() - start_time, 2)}s")
    logging.info(f"Tempo de Treinamento DQN: {round(time.time() - start_time, 2)}s")
    logging.info(f'Fim treinamento do algoritmo DQN')


    logging.info(f'Iniciando avaliação do algoritmo DQN conjunto de teste')
    accuracy, dqn_model = agente.env3W_dqn_eval(dataset_test_scaled, n_envs = 1)
    print(f'Acurácia de {accuracy * 100:.2f}% no conjunto de dados de teste usando DQN')
    logging.info(f'Acurácia de {accuracy:.5f} no conjunto de dados de teste usando DQN')
    logging.info(f'Fim avaliação  do algoritmo DQN conjunto de teste')'''
    
    start_time = time.time()
    agente = Agent(path_model)
    agente.env3W_ppo(dataset_train_scaled, n_envs = 5)  
    print(f"Tempo de Treinamento PPO: {round(time.time() - start_time, 2)}s")
    logging.info(f"Tempo de Treinamento PPO: {round(time.time() - start_time, 2)}s")
    logging.info(f'Fim treinamento do algoritmo PPO')

    logging.info(f'Iniciando avaliação do algoritmo PPO conjunto de teste')
    accuracy, ppo_model = agente.env3W_ppo_eval(dataset_test_scaled, n_envs = 1)
    print(f'Acurácia de {accuracy * 100:.2f}% no conjunto de dados de teste usando PPO')
    logging.info(f'Acurácia de {accuracy:.5f} no conjunto de dados de teste usando PPO')
    logging.info(f'Fim avaliação  do algoritmo PPO conjunto de teste')

    logging.info(f'Iniciando a validação do modelo PPO') 
    validation_dqn(accuracy, dataset_validation_scaled, ppo_model, event_name)

    logging.info(f'Concluído a execução do aprendizado por reforço')
    
    

Número de amostras da classe 0: 8430978
Número de amostras da classe 4: 2460270


In [4]:
start_time = time.time()
agente = Agent(path_model)
agente.env3W_ppo(dataset_train_scaled, n_envs = 5)  
print(f"Tempo de Treinamento PPO: {round(time.time() - start_time, 2)}s")
logging.info(f"Tempo de Treinamento PPO: {round(time.time() - start_time, 2)}s")
logging.info(f'Fim treinamento do algoritmo PPO')




c:\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
Logging to ..\models\Flow Instability\tensorboard_logs\PPO_2
------------------------------
| time/              |       |
|    fps             | 6613  |
|    iterations      | 1     |
|    time_elapsed    | 1     |
|    total_timesteps | 10240 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1778        |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 20480       |
| train/                  |             |
|    approx_kl            | 0.018178323 |
|    clip_fraction        | 0.384       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.676      |
|    explained_variance   | -0.0103     |
|    learning_rate        | 0.001       |
|    loss                 | 2.33        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0405     |
|    value_loss           | 5.89 

In [5]:
logging.info(f'Iniciando avaliação do algoritmo PPO conjunto de teste')
accuracy, ppo_model = agente.env3W_ppo_eval(dataset_test_scaled, n_envs = 1)
print(f'Acurácia de {accuracy * 100:.2f}% no conjunto de dados de teste usando PPO')
logging.info(f'Acurácia de {accuracy:.5f} no conjunto de dados de teste usando PPO')
logging.info(f'Fim avaliação  do algoritmo PPO conjunto de teste')

c:\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Acurácia de 94.61% no conjunto de dados de teste usando PPO


In [6]:
logging.info(f'Iniciando a validação do modelo PPO') 
validation_dqn(accuracy, dataset_validation_scaled, ppo_model, event_name)

logging.info(f'Concluído a execução do aprendizado por reforço')

Acurácia da 0ª instância: 100.000%
Acurácia da 1ª instância: 28.708%
Acurácia da 2ª instância: 80.999%
Acurácia da 3ª instância: 100.000%
Acurácia da 4ª instância: 100.000%
Acurácia da 5ª instância: 100.000%
Acurácia da 6ª instância: 100.000%
Acurácia da 7ª instância: 100.000%
Acurácia da 8ª instância: 67.682%
Acurácia da 9ª instância: 100.000%
Acurácia da 10ª instância: 100.000%
Acurácia da 11ª instância: 100.000%
Acurácia da 12ª instância: 100.000%
Acurácia da 13ª instância: 100.000%
Acurácia da 14ª instância: 100.000%
Acurácia da 15ª instância: 54.755%
Acurácia da 16ª instância: 0.000%
Acurácia da 17ª instância: 0.000%
Acurácia da 18ª instância: 100.000%
Acurácia da 19ª instância: 50.154%
Acurácia da 20ª instância: 100.000%
Acurácia da 21ª instância: 100.000%
Acurácia da 22ª instância: 100.000%
Acurácia da 23ª instância: 100.000%
Acurácia da 24ª instância: 100.000%
Acurácia da 25ª instância: 98.802%
Acurácia da 26ª instância: 75.682%
Acurácia da 27ª instância: 0.000%
Acurácia da 28ª